In [1]:
import requests
import pandas as pd

from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer

c:\Users\stu83\miniconda3\envs\dacon\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
csv_name = 'target_protein_wiki'
df = pd.read_csv(csv_name + '.csv')

In [3]:
# 임베딩 모델 설정 (예: Sentence-BERT 사용)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\stu83\miniconda3\envs\dacon\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def get_wiki_content(url):
    """ 주어진 Wikipedia URL에서 내용을 크롤링하여 텍스트를 반환합니다. """
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        content = ' '.join([para.text for para in paragraphs])
        return content.strip()
    except Exception as e:
        return f"Error: {str(e)}"

In [5]:
# Wikipedia URL이 있는 경우 크롤링하고 임베딩을 생성
def generate_wiki_embedding(row):
    if pd.notna(row['Wiki']):
        content = get_wiki_content(row['Wiki'])
        if "Error" not in content:
            embedding = embedding_model.encode(content)
            return embedding
        else:
            return None
    return None

In [6]:
# "Wiki" 컬럼에서 값이 "FALSE"인 행들을 제거합니다.
df_cleaned = df[df['Wiki'] != "FALSE"]

# 결과를 확인하거나 저장할 수 있습니다.
print(df_cleaned)

                                          Target Name  \
1             Macrophage-stimulating protein receptor   
2                           Cyclin-dependent kinase 7   
4                                   Plasma kallikrein   
5                       PI3-kinase p110-alpha subunit   
6   Signal transducer and activator of transcripti...   
7                     Rho-associated protein kinase 2   
8                        Tyrosine-protein kinase JAK2   
10              Serine/threonine-protein kinase B-raf   
11                        Tyrosine-protein kinase TXK   
12                      PI3-kinase p110-delta subunit   
14               Serine/threonine-protein kinase mTOR   
15                Dual specificity protein kinase TTK   
16                    Tyrosine-protein kinase ITK/TSK   
17                               Choline kinase alpha   
18                        Tyrosine-protein kinase TEC   
19              Tyrosine-protein kinase receptor FLT3   
20                          Cyc

In [7]:
# "Wiki" 컬럼에서 값이 "FALSE"인 행들을 제거합니다.
df_cleaned = df[df['Wiki'] != "FALSE"].copy()  # .copy()를 사용하여 슬라이스의 복사본을 만듭니다.

# "Wiki_Document" 컬럼을 생성하여 크롤링한 내용을 추가합니다.
df_cleaned.loc[:, 'Wiki_Document'] = df_cleaned.apply(lambda row: get_wiki_content(row['Wiki']), axis=1)

# 임베딩 값 저장
df_cleaned['Wiki_Embedding'] = df_cleaned.apply(generate_wiki_embedding, axis=1)

In [8]:
# 최종 데이터프레임을 CSV 파일로 저장
df_cleaned.to_csv('target_protein_embeddings.csv', index=False)
print("Final dataframe with Wiki Embeddings saved to 'final_train_data_with_embeddings.csv'")

Final dataframe with Wiki Embeddings saved to 'final_train_data_with_embeddings.csv'
